In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("sales_data.csv",index_col=0)

In [3]:
df.head()

,sku,product_line,brand,sales,price
SKU_ID,,,,,
1,Samsung Stereo Headset Wird EO-EG920BWEG,Phones Mobile Accessories,SAMSUNG\r,1,1359.0
2,HITACHI REF SBS RM-700AGPND4X-(DIA),Refrigerators,HITACHI\r,1,184408.0
3,SIEMENS DISHWASHER SN26L801IN,Dishwashers,SIEMENS\r,1,34425.0
4,Super General S/AC 1.5T SGSI185-3BE 3S,Air Conditioners,Super General\r,1,29750.0
5,Apple 12W USB Power Adapter,Tablets & Detachables,APPLE\r,1,1700.0


In [4]:
df['brand'] = df["brand"].str.replace('\r','')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 1 to 2015
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sku           2015 non-null   object 
 1   product_line  2015 non-null   object 
 2   brand         2015 non-null   object 
 3   sales         2015 non-null   int64  
 4   price         2015 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 94.5+ KB


In [6]:
df.brand.sort_values().unique()

array(['A.O.SMITH', 'AA', 'ACER', 'ADATA', 'AIRPLUS', 'AIRTEL',
       'ALTEC LANSING', 'AMAZON', 'APPLE', 'AQUA SURE', 'AQUAGUARD',
       'ASUS', 'BAJAJ', 'BEATS', 'BELKIN', 'BISSELL', 'BLUESTAR', 'BOAT',
       'BOOK MY SHOW', 'BOOMPODS', 'BOSCH', 'BOSE', 'BRAUN', 'BRILYANT',
       'CANON', 'CAPDASE', 'CARRIER', 'CASEMATE', 'CIRCLE', 'CLEARTRIP',
       'CONTISTAR', 'CREATIVE', 'CROMA', 'CROMA-ADONIS', 'Cyber Power',
       'DAIKIN', 'DEFUNC', 'DELL', 'DLINK', 'DRAV', 'DURACELL', 'Digisol',
       'EDU SCIENCE', 'ELGI', 'ENERGIZER', 'ENVENT', 'ENZATEC',
       'EUREKAFORB', 'EVEREADY', 'FABER', 'FENDA', 'FITBIT',
       'FUSE CHICKEN', 'GARMIN', 'GIONEE', 'GM', 'GO PRO', 'GODREJ',
       'GOOGLE', 'GPO', 'Gigaset', 'HAMA', 'HINDWARE', 'HITACHI',
       'HONEYWELL', 'HONOR', 'HP', 'HTC', 'HUAWEI', 'Hitachi-TV', 'IBALL',
       'IDEAL', 'IFB', 'INTEL', 'ITEK', 'JABRA', 'JBL', 'KENT', 'KFC',
       'KOTION', 'Kindle', 'LAVA', 'LENOVO', 'LG', 'LOGITECH', 'Livpure',
       'MARC', 'MCCO

In [87]:
from fuzzywuzzy import fuzz
from nltk.stem import WordNetLemmatizer

wl = WordNetLemmatizer()

brand_alias={
    r'morphyrich':['morphy'],
    r'motorola':['moto'],
    r'hitachitv':['hitachi'],
    r'cromaadonis':['croma'],
    r'brilyant':['catz'],
    r'microsoft':[r'\bms\b'],
    r'sony':[r'ps4'],
    r'ubisoft':[r'ps4.*game'],
    r'natgeo':[r'nationalgeographic']
}

def brand_matcher(query,brands,brand_alias):

    res = set()
    #matching exact brand name
    search_query = re.sub(r'\-|\'|e-','',query.lower())
    for brand in brands:
        
        search_brand = re.sub(r'\-|\'','',brand.lower())
        
        if re.search(r'\b'+search_brand+r'\b',search_query):
            
            res.add((brand,100))
            
        if search_brand in brand_alias and re.search('|'.join(r'\b'+sim+r'\b' for sim in brand_alias[search_brand]),search_query):
            res.add((brand,100))
    
    #matching without spaces
    search_query = re.sub(r'\-|\'|e-','',query.lower())
    
    for brand in brands:
        
        search_brand= re.sub(r'\-|\'| ','',brand.lower())
        score = fuzz.partial_ratio(search_brand,search_query)
        
        if search_brand in brand_alias:
            for alias in brand_alias[search_brand]:
                score = max(score,fuzz.partial_ratio(alias,search_query))
        
        res.add((brand,score))
           
    return sorted(res,reverse=True,key=lambda x: x[1])[:3]
        
        

In [88]:

for data in df.values:
    res = brand_matcher(data[0],df.brand.unique(),brand_alias)
    
    if res and data[2] not in [brand[0] for brand in res]:
        print(data)
        print(res)

['Tizum Prtbl Travel Accessories Carry Bag' 'Phones Mobile Accessories'
 'AIRPLUS' 4 899.0]
[('ULTRA', 80), ('DRAV', 75), ('ACER', 75)]
['Ahha Smrt Flipcase Zaki iPad Air BLK/RED' 'Tablets & Detachables'
 'CAPDASE' 1 220.0]
[('AIRTEL', 67), ('JBL', 67), ('CASEMATE', 62)]
['Casemate Naked Tough Case iPhone 6/6S' 'Phones Mobile Accessories'
 'STUFFCOOL' 1 1166.0]
[('CASEMATE', 100), ('SEAGATE', 71), ('GPO', 67)]
['Ducati Fighter Back case iPhone 7 Blk' 'Phones Mobile Accessories'
 'SKYWATER' 1 1890.0]
[('BRILYANT', 75), ('GPO', 67), ('HTC', 67)]
['SoundLink Color BT Speaker White KR' 'Shop in Shop' 'BOSE' 1 7920.0]
[('DLINK', 100), ('SONY', 75), ('TP-LINK', 67)]
['Hwell HDMI to Mini Display port cbl WHT' 'Peripherals' 'HONEYWELL' 1
 2099.0]
[('DELL', 75), ('GPO', 67), ('HTC', 67)]
['Stuffcool Puretuff Tempered iPhone 7+' 'Phones Mobile Accessories'
 'CASEMATE' 4 436.5]
[('STUFFCOOL', 100), ('PUREIT', 83), ('Livpure', 71)]
['Hama Mini HDMI AtoC 2m 74229' 'Peripherals' 'CROMA' 5 194.0]
[('

In [9]:
df.product_line.unique()

array(['Phones Mobile Accessories', 'Refrigerators', 'Dishwashers',
       'Air Conditioners', 'Tablets & Detachables', 'Gaming Software',
       'Mobile Computing', 'Kitchen Appliances', 'MP3 Players',
       'Input Devices', 'Earphones/Headphones', 'Home Appliances',
       'Washing Machines', 'Water Purifiers', 'Microwaves',
       'Travel Accessories', 'Phones Fixed', 'Smart Phones (OS Based)',
       'Computer Bags', 'Shop in Shop', 'Health & Personal Care',
       'Home Theatre', 'Phones Mobile', 'Lifestyle & Wearables',
       'Digital Cameras', 'Security', 'TV LCD', 'Peripherals',
       'Storage Media', 'Printers & Office Solutions', 'Gaming Hardware',
       'Imaging Accessories', 'Speakers', 'PC Accessories',
       'Phones Mobile Providers', 'Cookware', 'Fans',
       'Entertainment Accessories', 'Desktops', 'DVD/VCD/Content', 'Toys',
       'Networking', 'Dummy Laptop / Netbook / Tabs',
       'Cooling & Heating Appliances', 'Ovens & Cookers', 'Power',
       'Air Purifier

In [10]:
ana_grp = df.groupby(df['product_line'])['sku'].apply(lambda x:x.to_list())

In [11]:
ana_grp.index

Index(['Air Conditioners', 'Air Purifier', 'Audio Systems', 'Camcorders',
       'Computer Bags', 'Connected Homes & Housewares', 'Cookware',
       'Cooling & Heating Appliances', 'DVD/VCD/Content', 'Desktops',
       'Digital Cameras', 'Dishwashers', 'Dryers',
       'Dummy Laptop / Netbook / Tabs', 'ECharge', 'Earphones/Headphones',
       'Entertainment Accessories', 'Fans', 'Gaming Hardware',
       'Gaming Software', 'Health & Personal Care', 'Home Appliances',
       'Home Theatre', 'Imaging Accessories', 'Input Devices',
       'Kitchen Appliances', 'Lifestyle & Wearables', 'MP3 Accessories',
       'MP3 Players', 'Microwaves', 'Mobile Computing', 'Networking',
       'Other Charges', 'Ovens & Cookers', 'PC Accessories', 'Peripherals',
       'Phone Brand Attachments', 'Phones Fixed', 'Phones Mobile',
       'Phones Mobile Accessories', 'Phones Mobile Providers', 'Power',
       'Printers & Office Solutions', 'Refrigerators', 'Security',
       'Shop in Shop', 'Simcard', 'Smart

In [12]:
from nltk.stem import WordNetLemmatizer

unexp_mapper = {
    r'air conditioner':['ac'],
    r'washing machine':['wm'],
    r'desktop':['pc'],
    r'gaming hardware':['game'],
    r'gaming software':['game'],
    r'computer bag':['backpack','carrycase','bag'],
    r'earphone|headphone':['headset','earbud'],
    r'tv lcd':['tv','lcd','led','television'],
    r'smart phone':['mobile phone','phone mobile'],
    r'(phone mobile)|(mobile phone)':['smart phone','mobile phone','phone mobile'],
}
cat_alias = {}
wl = WordNetLemmatizer()


for cat in ana_grp.index:
    temp = [val.strip() for val in re.split(r'&|/',cat)]

    if len(temp[0].split(' '))>1:
        prefix = temp[0].split(' ')[0]
        for i in range(1,len(temp)):
            temp[i] = prefix+' '+temp[i]
            
    elif len(temp[-1].split(' '))>1:
        suffix = temp[-1].split(' ')[-1]
        for i in range(len(temp)-1):
            temp[i] += ' '+suffix
    
    cat_alias[cat] = []
    if cat in unexp_mapper:
        cat_alias[cat].extend(unexp_mapper[cat])
        
    for alias in  temp:
        
        root_form = ' '.join(wl.lemmatize(word.lower()) for word in alias.split(' '))
        if re.search(r'\(.*\)',root_form):
            cat_alias[cat].append(root_form)
            root_form = re.sub(r'\(.*\)','',root_form).strip()
        
        for key,values in unexp_mapper.items():
            if re.search(key,root_form):
                for value in values:
                    cat_alias[cat].append(re.sub(key,value,root_form).strip())
            
        cat_alias[cat].append(root_form)
        
        

In [13]:
cat_alias

{'Air Conditioners': ['ac', 'air conditioner'],
 'Air Purifier': ['air purifier'],
 'Audio Systems': ['audio system'],
 'Camcorders': ['camcorder'],
 'Computer Bags': ['backpack', 'carrycase', 'bag', 'computer bag'],
 'Connected Homes & Housewares': ['connected home', 'connected housewares'],
 'Cookware': ['cookware'],
 'Cooling & Heating Appliances': ['cooling appliance', 'heating appliance'],
 'DVD/VCD/Content': ['dvd', 'vcd', 'content'],
 'Desktops': ['pc', 'desktop'],
 'Digital Cameras': ['digital camera'],
 'Dishwashers': ['dishwasher'],
 'Dryers': ['dryer'],
 'Dummy Laptop / Netbook / Tabs': ['dummy laptop',
  'dummy netbook',
  'dummy tab'],
 'ECharge': ['echarge'],
 'Earphones/Headphones': ['headset',
  'earbud',
  'earphone',
  'headset',
  'earbud',
  'headphone'],
 'Entertainment Accessories': ['entertainment accessory'],
 'Fans': ['fan'],
 'Gaming Hardware': ['game', 'gaming hardware'],
 'Gaming Software': ['game', 'gaming software'],
 'Health & Personal Care': ['health car

In [14]:
def cat_matcher(query,cats,cat_alias):
    
    res = []
    search_query = re.sub(r'\-|/|\&','',query.lower())
    search_query = re.sub(r' +',' ',search_query).strip()
    #exact match
    for cat in cats:
        
        if re.search(cat.lower(),search_query):
            res.append(cat)
            continue
            
        if re.search('|'.join(r'\b'+alias+r'\b' for alias in cat_alias[cat]),search_query):
            res.append((cat,len(cat)))

    if res:   
        return res
    
    # partial matching using partial ratio
    for cat in cats:
        score = 0
        for alias in cat_alias[cat]:
            score = max(score,fuzz.partial_ratio(alias,search_query)*len(alias)/100)
            
        res.append((cat,score))
        
    res.sort(reverse=True,key=lambda x:x[1])
    
    return res[:3]
         
    
    

In [15]:
brands= df.brand.unique()

query = 'smart phone accessory'
brand = brand_matcher(query,brands, brand_alias)
brand_df = df.loc[df['brand'].apply(lambda x: x in brand)] if brand else df
cat = {category[0] for category in cat_matcher(query,brand_df['product_line'].unique(),cat_alias)}

In [16]:
brand_df.loc[brand_df['product_line'].apply(lambda x: x in cat)]

,sku,product_line,brand,sales,price
SKU_ID,,,,,


In [337]:
from fuzzywuzzy import fuzz
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
import numpy as np

unexp_cat_alias = {
    r'air conditioner':['ac'],
    r'washing machine':['wm'],
    r'desktop':['pc'],
    r'gaming hardware':['game'],
    r'gaming software':['game'],
    r'computer bag':['backpack','carrycase','bag'],
    r'earphone|headphone':['headset','earbud'],
    r'tv lcd':['tv','lcd','led','television'],
    r'smart phone':['mobile phone','phone mobile'],
    r'(phone mobile)|(mobile phone)':['smart phone','mobile phone','phone mobile'],
}

unexp_brand_alias={
    r'morphyrich':['morphy'],
    r'motorola':['moto'],
    r'hitachitv':['hitachi'],
    r'cromaadonis':['croma'],
    r'brilyant':['catz'],
    r'sony':['ps4'],
    r'ubisoft':['ps4'],
    r'wb':['ps4']
}

class ProductMatcher:
    
    def __init__(self,df: pd.DataFrame,*args,**kwargs)->None:
        self.df = df
        self.brand_alias: dict[str,list[str]] = None
        self.cat_alias: dict[str,list[str]] = None
        
        if 'brand_alias' in kwargs:
            self.brand_alias = kwargs['brand_alias']
        
        if 'cat_alias' in kwargs:
            self.cat_alias = self.create_cat_alias(kwargs['cat_alias'])
        
    def brand_matcher(self,query:str)->list[tuple[str,int]]:
        
        brands = self.df['brand'].unique()
        brand_alias = self.brand_alias
        res = set()
        #matching exact brand name
        search_query = re.sub(r'\-|\'|e-','',query.lower())
        for brand in brands:

            search_brand = re.sub(r'\-|\'','',brand.lower())

            if re.search(r'\b'+search_brand+r'\b',search_query):

                res.add((brand,1))
                continue

            if search_brand in brand_alias and\
            re.search('|'.join(r'\b'+sim+r'\b' for sim in brand_alias[search_brand]),search_query):
                
                res.add((brand,1))

        #matching without spaces
        search_query = re.sub(r'\-|\'|e-','',query.lower())

        for brand in brands:

            search_brand= re.sub(r'\-|\'| ','',brand.lower())
            score = fuzz.partial_ratio(search_brand,search_query)/100

            if search_brand in brand_alias:
                for alias in brand_alias[search_brand]:
                    score = max(score,fuzz.partial_ratio(alias,search_query)/100)

            res.add((brand,score))
  
        return sorted(res,reverse=True,key=lambda x: x[1])[:3]
    
    def cat_matcher(self,query,brand_filter:set[str])->list[tuple[str,int]]:
        if brand_filter:
            brand_df = self.df.loc[self.df['brand'].apply(lambda x: x in brand_filter)]
        else:
            brand_df = self.df
        cats = brand_df['product_line'].unique()
        res = set()
        search_query = re.sub(r'\-|/|\&','',query.lower())
        search_query = re.sub(r' +',' ',search_query).strip()
        #exact match
        for cat in cats:

            if re.search(r'\b'+cat.lower()+r'\b',search_query):
                res.add((cat,1))
                continue

            if re.search('|'.join(r'\b'+alias+r'\b' for alias in cat_alias[cat]),search_query):
                res.add((cat,1))
                continue
            
            for token in search_query.split(' '):
                    
                if not(re.search(r'[aeiou]',token) and len(token)>1):
                    continue
         
                if cat.lower().startswith(token):
                    res.add((cat,1))
                    break
                

        # partial matching using partial ratio
        for cat in cats:
            score = 0
            for alias in cat_alias[cat]:
                score = max(score,fuzz.partial_ratio(alias,search_query)/100)
            
            res.add((cat,score))
      
        return sorted(res,reverse=True,key=lambda x:x[1])[:3]
    
    def create_cat_alias(self,unexp_cat_alias:dict[str,list[str]]):
        
        cat_alias = {}
        wl = WordNetLemmatizer()


        for cat in ana_grp.index:
            temp = [val.strip() for val in re.split(r'&|/',cat)]

            if len(temp[0].split(' '))>1:
                prefix = temp[0].split(' ')[0]
                for i in range(1,len(temp)):
                    temp[i] = prefix+' '+temp[i]

            elif len(temp[-1].split(' '))>1:
                suffix = temp[-1].split(' ')[-1]
                for i in range(len(temp)-1):
                    temp[i] += ' '+suffix

            cat_alias[cat] = []
            if cat in unexp_cat_alias:
                cat_alias[cat].extend(unexp_cat_alias[cat])

            for alias in  temp:

                root_form = ' '.join(wl.lemmatize(word.lower()) for word in alias.split(' '))
                if re.search(r'\(.*\)',root_form):
                    cat_alias[cat].append(root_form)
                    root_form = re.sub(r'\(.*\)','',root_form).strip()

                for key,values in unexp_cat_alias.items():
                    if re.search(key,root_form):
                        for value in values:
                            cat_alias[cat].append(re.sub(key,value,root_form).strip())

                cat_alias[cat].append(root_form)
                
        self.cat_alias = cat_alias
    
    def sku_search(self,query:str,brand_filter:set[str],cat_filter:set[str])->pd.Index:
        
        search_space = df.loc[
            df.apply(
                lambda x: x['brand'] in brand_filter and x['product_line'] in cat_filter,
                axis=1
            )
        ]
    
        search_space['sku'] = search_space['sku'].apply(
            lambda x: re.sub(
                r' +',' ',
                re.sub(r'\-|/|\&','',x.lower())
            ).strip()
        )
        
        query = re.sub(
            r' +',' ',
            re.sub(r'\-|/|\&','',query.lower())
        ).strip()
        
        
        n = len(query.split(' '))
        threshold = (n-1)/n
        
        total = 0
        
        def score(str1,str2):
            tokens = str1.split(' ')
            total = 0
            for token in tokens:
                total += fuzz.partial_ratio(token,str1)
            
            return total/len(tokens)
            
        
        return search_space.loc[
            search_space['sku'].apply(
                lambda sku: score(query,sku)>= threshold
            )
        ].index
        
        
        
    
    def state_space_search(self,query:str,mini_fetch:int)->pd.DataFrame:
        import heapq
        
        brands = self.brand_matcher(query)
        brand_cat_q = []
            
        for brand,brand_score in brands:
            cats = self.cat_matcher(query,{brand,})
            for cat,cat_score in cats:
                heapq.heappush(brand_cat_q,(-brand_score*cat_score,(brand,cat)))
        
        res_idx = []
    
        while(brand_cat_q and len(res_idx)<mini_fetch):
            score,(brand,cat) = heapq.heappop(brand_cat_q)
            res_idx.extend(self.sku_search(query,{brand,},{cat,}))
         
        return self.df.iloc[np.array(res_idx)-1]
            
            

In [338]:
df = pd.read_csv('sales_data.csv',index_col=0)
df['brand'] = df["brand"].str.replace('\r','')
pm = ProductMatcher(df,brand_alias = unexp_brand_alias,cat_alias=unexp_cat_alias)

In [339]:
import warnings
warnings.filterwarnings("ignore")
pm.state_space_search("bajaj cooler",9)

,sku,product_line,brand,sales,price
SKU_ID,,,,,
1504,Bajaj OTG 2200TMSS,Ovens & Cookers,BAJAJ,2,4955.000000
1623,Bajaj Rice Cooker 1.8L Majsty RCX18 Plus,Ovens & Cookers,BAJAJ,2,2129.000000
541,Bajaj R/Cooler DC2015 ICON,Cooling & Heating Appliances,BAJAJ,1,8393.000000
416,Bajaj ELX 36 LED Emergency Light,Home Appliances,BAJAJ,2,2049.000000
987,Bajaj Geyser Flora 3Ltr/4.5Kw,Home Appliances,BAJAJ,1,3890.000000
1496,Bajaj Geyser Flora 3Ltr/3Kw,Home Appliances,BAJAJ,4,3676.750000
1536,Bajaj Emergency Light LED ELX16 Blue,Home Appliances,BAJAJ,3,1549.000000
596,Google Chromecast 2 Black,DVD/VCD/Content,GOOGLE,88,3000.136364
260,Google Pixel XL LTE 32GB Silver,Smart Phones (OS Based),GOOGLE,1,47775.000000


In [340]:
matched = 0
for idx,data in zip(df.index,df.values):
    res = pm.state_space_search(data[0],9)
    if idx in res.index:
        matched += 1
        
    

In [341]:
matched/df.shape[0]

0.6486352357320099

In [342]:
pm.state_space_search('ps4 game gta V',9)

,sku,product_line,brand,sales,price
SKU_ID,,,,,
88,PS4 DualShock Controller Black,Gaming Hardware,SONY,3,4823.666667
952,SONY PS4 Slim 1TB Tekken 7,Gaming Hardware,SONY,1,32990.000000
1156,Sony PS4 Slim 500GB + Controller,Gaming Hardware,SONY,1,29990.000000
1164,SONY PS4 Slim 500GB HZD + R&C + DC,Gaming Hardware,SONY,6,28990.000000
1594,SONY PS4 Slim 1TB UCC + TLOU + GOW,Gaming Hardware,SONY,1,32990.000000
1624,Sony PS4 SLIM 1TB UCC,Gaming Hardware,SONY,9,31634.444440
1868,PS4 Dual Shock Controller Grey Blue,Gaming Hardware,SONY,1,4491.000000
7,PS4 Game Assassins Creed Syndicate,Gaming Software,UBISOFT,1,1299.000000
65,PS4 Game Assassins Creed 4 Black Flag,Gaming Software,UBISOFT,1,1299.000000


In [343]:
df.loc[df['sku'].str.contains('PS4')]

,sku,product_line,brand,sales,price
SKU_ID,,,,,
7,PS4 Game Assassins Creed Syndicate,Gaming Software,UBISOFT,1,1299.000000
65,PS4 Game Assassins Creed 4 Black Flag,Gaming Software,UBISOFT,1,1299.000000
88,PS4 DualShock Controller Black,Gaming Hardware,SONY,3,4823.666667
772,PS4 Game GTA V,Gaming Software,WB,1,2659.000000
856,PS4 Game Watch Dogs Complete Edition,Gaming Software,UBISOFT,1,1299.000000
952,SONY PS4 Slim 1TB Tekken 7,Gaming Hardware,SONY,1,32990.000000
1156,Sony PS4 Slim 500GB + Controller,Gaming Hardware,SONY,1,29990.000000
1164,SONY PS4 Slim 500GB HZD + R&C + DC,Gaming Hardware,SONY,6,28990.000000
1270,PS4 Game Mortal Kombat XL,Gaming Software,WARNER,2,1499.000000
